In [47]:
import re
import requests
from curl_cffi import requests as curl_r
import json
import pandas as pd

BASE_URL = 'https://www.zillow.com/async-create-search-page-state'

cookies = {
    '_hp2_id.1215457233': '%7B%22userId%22%3A%222230556399004484%22%2C%22pageviewId%22%3A%227996225023036108%22%2C%22sessionId%22%3A%222432367684716351%22%2C%22identity%22%3Anull%2C%22trackerVersion%22%3A%224.0%22%7D',
    'zguid': '24|%24bfec1c83-95a4-4ebd-a83f-ed8c3e32a6f0',
    'zgsession': '1|7a7f86a6-ec5c-42db-9958-0ccf54c505b0',
    'zjs_anonymous_id': '%22bfec1c83-95a4-4ebd-a83f-ed8c3e32a6f0%22',
    'zjs_user_id': 'null',
    'zg_anonymous_id': '%224e64cb82-f529-43d3-b1d0-bc0fbc6e91f9%22',
    'pxcts': '37af3558-cf4e-11ef-8b7d-a53cf70d2449',
    '_pxvid': '37af2b21-cf4e-11ef-8b7c-f8ee72dc674e',
    'JSESSIONID': 'DC22B5374647DD324B067741B28D1686',
    'AWSALB': 'mTS09LI7gjnccILOJGu990b/YGRi0Ox7PsLrHk5Qn6o+RINgNEmy1pTmPwGSHf1W2cN2SBdF5pyzsXKazTGX+6h/EOEz3QLXcncl/2XdwNXjUgXpv6zj12f8jK/X',
    'AWSALBCORS': 'mTS09LI7gjnccILOJGu990b/YGRi0Ox7PsLrHk5Qn6o+RINgNEmy1pTmPwGSHf1W2cN2SBdF5pyzsXKazTGX+6h/EOEz3QLXcncl/2XdwNXjUgXpv6zj12f8jK/X',
    'search': '6|1739110371077%7Crect%3D46.807656290966555%2C-55.405427593750005%2C21.626896832680362%2C-104.31656040625%26rid%3D36%26disp%3Dmap%26mdm%3Dauto%26p%3D7%26listPriceActive%3D1%26fs%3D1%26fr%3D0%26mmm%3D0%26rs%3D0%26singlestory%3D0%26housing-connector%3D0%26parking-spots%3Dnull-%26abo%3D0%26garage%3D0%26pool%3D0%26ac%3D0%26waterfront%3D0%26finished%3D0%26unfinished%3D0%26cityview%3D0%26mountainview%3D0%26parkview%3D0%26waterview%3D0%26hoadata%3D1%26zillow-owned%3D0%263dhome%3D0%26showcase%3D0%26featuredMultiFamilyBuilding%3D0%26onlyRentalStudentHousingType%3D0%26onlyRentalIncomeRestrictedHousingType%3D0%26onlyRentalMilitaryHousingType%3D0%26onlyRentalDisabledHousingType%3D0%26onlyRentalSeniorHousingType%3D0%26commuteMode%3Ddriving%26commuteTimeOfDay%3Dnow%09%0936%09%7B%22isList%22%3Atrue%2C%22isMap%22%3Atrue%7D%09%09%09%09%09',
    '_px3': '98a3e76505f07bef738a88771760b9d5574c27eaed6d58c626e69690baf528c6:l72iNQfhnd40Q39XE5Cm4vXKsqUyhGA2Lu1WzHuUGCBdzmOzKx41qq9hyoFyShm/E7vSZB4dSCIkXjIvykofjw==:1000:kDmZRAiqyArRJTdeOuPqY6r2RtDCwcVj/20LDTJa5AWwHOz9+N70mBH9Yf6rKyZe27poq/udMECPQAm341vo15tfsp+6v8kTPJuqRd6n7I4zWhYlgFnk68N0q9mSOBlYE4JYuqzEJT5pEsmFz1acpSjc2gECfiBRdqrLfv/ctSKZ3zi9T0tgy6X+qTEnKhIXIRmQNhhns2LmWEnqoUQ0TEg93iZsgNgfhh7zMqk5Cmo=',
    '_dd_s': 'rum=0&expire=1736519409316',
}

headers = {
    'accept': '*/*',
    'accept-language': 'en-US,en;q=0.9,ar;q=0.8',
    'cache-control': 'no-cache',
    'content-type': 'application/json',
    'origin': 'https://www.zillow.com',
    'pragma': 'no-cache',
    'priority': 'u=1, i',
    'referer': 'https://www.zillow.com/nc/?searchQueryState=%7B%22pagination%22%3A%7B%7D%2C%22isMapVisible%22%3Atrue%2C%22mapBounds%22%3A%7B%22west%22%3A-85.9748856015625%2C%22east%22%3A-73.7471023984375%2C%22south%22%3A33.91154747652504%2C%22north%22%3A36.43463209815537%7D%2C%22mapZoom%22%3A7%2C%22usersSearchTerm%22%3A%22NC%22%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A36%2C%22regionType%22%3A2%7D%5D%2C%22filterState%22%3A%7B%22sort%22%3A%7B%22value%22%3A%22globalrelevanceex%22%7D%7D%2C%22isListVisible%22%3Atrue%7D',
    'sec-ch-ua': '"Google Chrome";v="131", "Chromium";v="131", "Not_A Brand";v="24"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36',
}

json_data = {
    'searchQueryState': {
        'pagination': {},
        'isMapVisible': True,
        'mapBounds': {
            'west': -85.9748856015625,
            'east': -73.7471023984375,
            'south': 33.91154747652504,
            'north': 36.43463209815537,
        },
        'mapZoom': 7,
        'usersSearchTerm': 'NC',
        'regionSelection': [
            {
                'regionId': 36,
                'regionType': 2,
            },
        ],
        'filterState': {
            'sortSelection': {
                'value': 'globalrelevanceex',
            },
            'isRecentlySold': {
                'value': True,
            },
            'isForSaleByAgent': {
                'value': False,
            },
            'isForSaleByOwner': {
                'value': False,
            },
            'isNewConstruction': {
                'value': False,
            },
            'isComingSoon': {
                'value': False,
            },
            'isAuction': {
                'value': False,
            },
            'isForSaleForeclosure': {
                'value': False,
            },
        },
        'isListVisible': True,
    },
    'wants': {
        'cat1': [
            'listResults',
            'mapResults',
        ],
    },
    'requestId': 6,
    'isDebugRequest': False,
}

responses = []

In [48]:
r = curl_r.put(BASE_URL, cookies=cookies, headers=headers, json=json_data, impersonate='chrome124')
assert r.status_code == 200, f'Request failed. status code: {r.status_code}'

try: 
    print("Found items:", len(r.json()['cat1']['searchResults']['mapResults']))
    responses.extend(r.json()['cat1']['searchResults']['mapResults'])
except KeyError:
    print("Non-existing key. Available keys are:\n", r.json().keys())    
print(len(responses))

Found items: 500
500


In [49]:
df = pd.DataFrame(responses)
df

,zpid,rawHomeStatusCd,marketingStatusSimplifiedCd,imgSrc,hasImage,detailUrl,statusType,statusText,price,priceLabel,...,minArea,buildingId,isBuilding,canSaveBuilding,unitCount,lotAreaString,lotId,badgeInfo,streetViewURL,streetViewMetadataURL
0,82766596,RecentlySold,RecentChange,https://photos.zillowstatic.com/fp/c7d8c5478f4...,True,/homedetails/1930-Harcourt-Cir-APT-103-Fayette...,SOLD,Sold,"$181,900",$182K,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,215427016,RecentlySold,RecentChange,https://photos.zillowstatic.com/fp/ac116e2a038...,True,/homedetails/89-Black-Cloud-Dr-Louisburg-NC-27...,SOLD,Sold,"$335,000",$335K,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,250202602,RecentlySold,RecentChange,https://photos.zillowstatic.com/fp/1c99550d4bc...,True,/homedetails/2120-Boulder-Ct-Gastonia-NC-28054...,SOLD,Sold,"$313,000",$313K,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5600637,RecentlySold,RecentChange,https://photos.zillowstatic.com/fp/8fe55aed0a7...,True,/homedetails/23-Auburndale-Dr-Asheville-NC-288...,SOLD,Sold,"$540,000",$540K,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5608582,RecentlySold,RecentChange,https://photos.zillowstatic.com/fp/7200b3bf30e...,True,/homedetails/71-Morris-St-Asheville-NC-28806/5...,SOLD,Sold,"$375,000",$375K,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,442397809,RecentlySold,RecentChange,https://photos.zillowstatic.com/fp/b43fe3f4947...,True,/homedetails/6810-Sawyer-Rd-1-Liberty-NC-27298...,SOLD,Sold,"$272,500",$273K,...,NaN,NaN,NaN,NaN,NaN,11.38 acres,NaN,NaN,NaN,NaN
496,442554080,RecentlySold,RecentChange,https://photos.zillowstatic.com/fp/19940878465...,True,/homedetails/322-Ladyfish-Loop-NW-LOT-25-Sunse...,SOLD,Sold,"$261,905",$262K,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
497,442679264,Pending,Non Owner Occupied,https://photos.zillowstatic.com/fp/17770def0fb...,True,/homedetails/267-Swinson-Dr-Dudley-NC-28333/44...,SOLD,Sold,"$245,900",$246K,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
498,442720124,RecentlySold,RecentChange,https://photos.zillowstatic.com/fp/cbb42a3b39b...,True,/homedetails/4604-4608-N-Church-St-Greensboro-...,SOLD,Sold,"$750,000",$750K,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
#pd.set_option('display.max_columns', None)

df.iloc[0:2]

,zpid,rawHomeStatusCd,marketingStatusSimplifiedCd,imgSrc,hasImage,detailUrl,statusType,statusText,price,priceLabel,...,builderName,isCdpResult,unitCount,isComingSoonCommunity,lotId,badgeInfo,buildingName,availabilityDate,lotAreaString,variableData
0,110510573,ForSale,For Sale by Agent,https://photos.zillowstatic.com/fp/fff79085be9...,True,/homedetails/2931-Judge-Manly-Dr-New-Bern-NC-2...,FOR_SALE,House for sale,"$270,000",$270K,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,51539344,ForSale,For Sale by Agent,https://photos.zillowstatic.com/fp/9f833da665a...,True,/homedetails/211-Trotters-Run-Jacksonville-NC-...,FOR_SALE,House for sale,"$265,000",$265K,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
responses[10]

{'zpid': '6177876',
 'rawHomeStatusCd': 'ForSale',
 'marketingStatusSimplifiedCd': 'For Sale by Agent',
 'imgSrc': 'https://photos.zillowstatic.com/fp/7f9012957b7e6435187425e135e6600b-p_e.jpg',
 'hasImage': True,
 'detailUrl': '/homedetails/1814-Berryhill-Rd-Charlotte-NC-28208/6177876_zpid/',
 'statusType': 'FOR_SALE',
 'statusText': 'House for sale',
 'price': '$750,000',
 'priceLabel': '$750K',
 'address': '1814 Berryhill Rd, Charlotte, NC 28208',
 'beds': 4,
 'baths': 3.0,
 'area': 2565,
 'latLong': {'latitude': 35.228783, 'longitude': -80.87906},
 'hdpData': {'homeInfo': {'zpid': 6177876,
   'streetAddress': '1814 Berryhill Rd',
   'zipcode': '28208',
   'city': 'Charlotte',
   'state': 'NC',
   'latitude': 35.228783,
   'longitude': -80.87906,
   'price': 750000.0,
   'bathrooms': 3.0,
   'bedrooms': 4.0,
   'livingArea': 2565.0,
   'homeType': 'SINGLE_FAMILY',
   'homeStatus': 'FOR_SALE',
   'daysOnZillow': -1,
   'isFeatured': False,
   'shouldHighlight': False,
   'zestimate': 

In [32]:
to_keep = ['address', 'price', 'beds', 'baths', 'area', 'detailUrl']
df = df[to_keep]
df

,address,price,beds,baths,area,detailUrl
0,"2931 Judge Manly Drive, New Bern, NC 28562","$270,000",3.0,2.0,1420.0,/homedetails/2931-Judge-Manly-Dr-New-Bern-NC-2...
1,"211 Trotters Run, Jacksonville, NC 28546","$265,000",3.0,2.0,1411.0,/homedetails/211-Trotters-Run-Jacksonville-NC-...
2,"26 Wildberry Court, Rocky Mount, NC 27804","$269,900",3.0,2.0,1440.0,/homedetails/26-Wildberry-Ct-Rocky-Mount-NC-27...
3,"882 Berea Church Rd, Elizabeth City, NC 27909","$270,000",3.0,2.0,1422.0,/homedetails/882-Berea-Church-Rd-Elizabeth-Cit...
4,"5413 Nc 42, Elm City, NC 27822","$258,000",3.0,2.0,1421.0,/homedetails/5413-State-Highway-42-E-Elm-City-...
...,...,...,...,...,...,...
497,"8528 Westhope St, Charlotte, NC 28216","$249,900",2.0,2.0,1075.0,/homedetails/8528-Westhope-St-Charlotte-NC-282...
498,"5850 Old Rural Hall Rd, Winston Salem, NC 27105","$249,000",3.0,2.0,1620.0,/homedetails/5850-Old-Rural-Hall-Rd-Winston-Sa...
499,"800 Hunters Way, Lexington, NC 27292","$595,000",4.0,3.0,3714.0,/homedetails/800-Hunters-Way-Lexington-NC-2729...
500,NaN,"From $299,000",NaN,NaN,NaN,/community/greater-charlotte-by-northway-homes...


In [33]:
help(df.drop_duplicates)

Help on method drop_duplicates in module pandas.core.frame:

drop_duplicates(subset: 'Hashable | Sequence[Hashable] | None' = None, *, keep: "Literal['first', 'last', False]" = 'first', inplace: 'bool' = False, ignore_index: 'bool' = False) -> 'DataFrame | None' method of pandas.core.frame.DataFrame instance
    Return DataFrame with duplicate rows removed.
    
    Considering certain columns is optional. Indexes, including time indexes
    are ignored.
    
    Parameters
    ----------
    subset : column label or sequence of labels, optional
        Only consider certain columns for identifying duplicates, by
        default use all of the columns.
    keep : {'first', 'last', False}, default 'first'
        Determines which duplicates (if any) to keep.
        - ``first`` : Drop duplicates except for the first occurrence.
        - ``last`` : Drop duplicates except for the last occurrence.
        - False : Drop all duplicates.
    inplace : bool, default False
        Whether to 

In [34]:
df = df.drop_duplicates()
df.shape

(502, 6)

In [35]:
help(pd.DataFrame.apply)

Help on function apply in module pandas.core.frame:

apply(self, func: 'AggFuncType', axis: 'Axis' = 0, raw: 'bool' = False, result_type: "Literal['expand', 'reduce', 'broadcast'] | None" = None, args=(), **kwargs)
    Apply a function along an axis of the DataFrame.
    
    Objects passed to the function are Series objects whose index is
    either the DataFrame's index (``axis=0``) or the DataFrame's columns
    (``axis=1``). By default (``result_type=None``), the final return type
    is inferred from the return type of the applied function. Otherwise,
    it depends on the `result_type` argument.
    
    Parameters
    ----------
    func : function
        Function to apply to each column or row.
    axis : {0 or 'index', 1 or 'columns'}, default 0
        Axis along which the function is applied:
    
        * 0 or 'index': apply function to each column.
        * 1 or 'columns': apply function to each row.
    
    raw : bool, default False
        Determines if row or column

In [36]:
df.loc[:, 'price'] = df.loc[:, 'price'].apply(lambda row: re.sub(r'[^\d]', '', row))
df

,address,price,beds,baths,area,detailUrl
0,"2931 Judge Manly Drive, New Bern, NC 28562",270000,3.0,2.0,1420.0,/homedetails/2931-Judge-Manly-Dr-New-Bern-NC-2...
1,"211 Trotters Run, Jacksonville, NC 28546",265000,3.0,2.0,1411.0,/homedetails/211-Trotters-Run-Jacksonville-NC-...
2,"26 Wildberry Court, Rocky Mount, NC 27804",269900,3.0,2.0,1440.0,/homedetails/26-Wildberry-Ct-Rocky-Mount-NC-27...
3,"882 Berea Church Rd, Elizabeth City, NC 27909",270000,3.0,2.0,1422.0,/homedetails/882-Berea-Church-Rd-Elizabeth-Cit...
4,"5413 Nc 42, Elm City, NC 27822",258000,3.0,2.0,1421.0,/homedetails/5413-State-Highway-42-E-Elm-City-...
...,...,...,...,...,...,...
497,"8528 Westhope St, Charlotte, NC 28216",249900,2.0,2.0,1075.0,/homedetails/8528-Westhope-St-Charlotte-NC-282...
498,"5850 Old Rural Hall Rd, Winston Salem, NC 27105",249000,3.0,2.0,1620.0,/homedetails/5850-Old-Rural-Hall-Rd-Winston-Sa...
499,"800 Hunters Way, Lexington, NC 27292",595000,4.0,3.0,3714.0,/homedetails/800-Hunters-Way-Lexington-NC-2729...
500,NaN,299000,NaN,NaN,NaN,/community/greater-charlotte-by-northway-homes...


In [37]:
df['link'] = df['detailUrl'].apply(lambda x: f"https://www.zillow.com/{x}")
df.loc[100, :]

address               250 Lake Eva Marie Dr, Raleigh, NC 27603
price                                                   349900
beds                                                       3.0
baths                                                      3.0
area                                                    1957.0
detailUrl    /homedetails/250-Lake-Eva-Marie-Dr-Raleigh-NC-...
link         https://www.zillow.com//homedetails/250-Lake-E...
Name: 100, dtype: object

In [38]:
df = df.drop(columns='detailUrl')
df

,address,price,beds,baths,area,link
0,"2931 Judge Manly Drive, New Bern, NC 28562",270000,3.0,2.0,1420.0,https://www.zillow.com//homedetails/2931-Judge...
1,"211 Trotters Run, Jacksonville, NC 28546",265000,3.0,2.0,1411.0,https://www.zillow.com//homedetails/211-Trotte...
2,"26 Wildberry Court, Rocky Mount, NC 27804",269900,3.0,2.0,1440.0,https://www.zillow.com//homedetails/26-Wildber...
3,"882 Berea Church Rd, Elizabeth City, NC 27909",270000,3.0,2.0,1422.0,https://www.zillow.com//homedetails/882-Berea-...
4,"5413 Nc 42, Elm City, NC 27822",258000,3.0,2.0,1421.0,https://www.zillow.com//homedetails/5413-State...
...,...,...,...,...,...,...
497,"8528 Westhope St, Charlotte, NC 28216",249900,2.0,2.0,1075.0,https://www.zillow.com//homedetails/8528-Westh...
498,"5850 Old Rural Hall Rd, Winston Salem, NC 27105",249000,3.0,2.0,1620.0,https://www.zillow.com//homedetails/5850-Old-R...
499,"800 Hunters Way, Lexington, NC 27292",595000,4.0,3.0,3714.0,https://www.zillow.com//homedetails/800-Hunter...
500,NaN,299000,NaN,NaN,NaN,https://www.zillow.com//community/greater-char...


In [39]:
df['price'] = df['price'].astype(int)

df.dtypes

address     object
price        int32
beds       float64
baths      float64
area       float64
link        object
dtype: object

In [40]:
df = df.fillna('-')

In [41]:
df

,address,price,beds,baths,area,link
0,"2931 Judge Manly Drive, New Bern, NC 28562",270000,3.0,2.0,1420.0,https://www.zillow.com//homedetails/2931-Judge...
1,"211 Trotters Run, Jacksonville, NC 28546",265000,3.0,2.0,1411.0,https://www.zillow.com//homedetails/211-Trotte...
2,"26 Wildberry Court, Rocky Mount, NC 27804",269900,3.0,2.0,1440.0,https://www.zillow.com//homedetails/26-Wildber...
3,"882 Berea Church Rd, Elizabeth City, NC 27909",270000,3.0,2.0,1422.0,https://www.zillow.com//homedetails/882-Berea-...
4,"5413 Nc 42, Elm City, NC 27822",258000,3.0,2.0,1421.0,https://www.zillow.com//homedetails/5413-State...
...,...,...,...,...,...,...
497,"8528 Westhope St, Charlotte, NC 28216",249900,2.0,2.0,1075.0,https://www.zillow.com//homedetails/8528-Westh...
498,"5850 Old Rural Hall Rd, Winston Salem, NC 27105",249000,3.0,2.0,1620.0,https://www.zillow.com//homedetails/5850-Old-R...
499,"800 Hunters Way, Lexington, NC 27292",595000,4.0,3.0,3714.0,https://www.zillow.com//homedetails/800-Hunter...
500,-,299000,-,-,-,https://www.zillow.com//community/greater-char...


In [42]:
df.to_csv('final_homes.csv', index=False)